In [1]:
import mand.core

from mand.core import Entity, node, Context

from mand.core import ObjectDb, _tr, Timestamp, Context
from mand.core import ProfileMonitor, PrintMonitor
from mand.lib.extrefdata import ExternalRefData, dataField
from mand.lib.workflow import Workbook, WorkItemOpenEvent, WorkItem
from mand.lib.portfolio import Portfolio
from mand.core import displayDict, displayMarkdown, displayListOfDicts, displayHeader
from mand.core import num, find
import datetime
from mand.lib.dbsetup import setUpDb

from mand.demos.trading import TradingBook, TradingPortfolio, MarketDataSource, MarketInterface

db = ObjectDb()
setUpDb(db)
db.describe()

<mand.db.ObjectDb object at 0x109f9dad0>: 106, mem=True, ro=False: entities=9, map=2


In [2]:
def makeTree(names):
    ret = []
    for name in names:
        subs = [ TradingBook(name+str(i)) for i in range(10) ]
        p = TradingPortfolio(name).write()
        p.setChildren(subs)
        ret.append(p)
    return ret

with db:
    pAll = TradingPortfolio('TopOfTheHouse').write()
    subs = makeTree(['Eq-Prop', 'Eq-Inst', ])# 'FX', 'Rates', 'Credit', 'Delta1', 'Loans', 'Commod', 'ETFs', 'Mtge'])
    pAll.setChildren(subs)
    
print pAll
print '# books:', len(pAll.books())
print '# children:', len(pAll.children())

# books: 20
# children: 2


In [3]:
with db:
    bExt  = _tr.TradingBook('Customer1')
    bExt2 = _tr.TradingBook('Customer2')
    
p1 = pAll.children()[0]
p2 = pAll.children()[1]

b1 = p1.children()[0]
b2 = p2.children()[0]

print bExt.meta.name()
print b1.meta.name()
print b2.meta.name()

Customer1
Eq-Prop0
Eq-Inst0


In [4]:
with db:
    s1_ibm  = MarketDataSource('source1.IBM')
    s1_goog = MarketDataSource('source1.GOOG')

s1_ibm.update(last=175.61)
s1_goog.update(last=852.12)

In [5]:
with db:
    ibm  = MarketInterface('IBM')
    goog = MarketInterface('GOOG')

In [6]:
with db:
    TradeOpenEvent = _tr.TradeOpenEvent
    cf1 = _tr.ForwardCashflow()
    ins1 = _tr.Equity()
    ins2 = _tr.Equity(assetName='GOOG.Eq.1')
    
    ts1 = Timestamp()
    
    ev1 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.65,
                         book1=b1,
                         book2=bExt).write()
    
    ts2 = Timestamp()
    
    s1_ibm.update(last=175.64)
    
    ts3 = Timestamp()
    
    ev2 = TradeOpenEvent(action='Buy',
                         item=ins2,
                         quantity=300,
                         premium=cf1,
                         unitPrice=852.12,
                         book1=b2,
                         book2=bExt).write()
    
    ev3 = TradeOpenEvent(action='Sell',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.85,
                         book1=b2,
                         book2=bExt2).write()
    
    ts4 = Timestamp()
    
    s1_ibm.update(last=175.70)
    s1_goog.update(last=852.11)
    
    ts5 = Timestamp()
    
    s1_ibm.update(last=175.68)
    s1_goog.update(last=852.13)
    
    eod = Timestamp()
    
    ev4 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.69,
                         book1=b1,
                         book2=bExt,
                         amends=ev1,
                         message='Sorry, the broker says you actually paid 69. signed: the middle office'
                        ).write(validTime=ev1.meta._timestamp.validTime)
    
    s1_ibm.update(last=177.68)
    s1_goog.update(last=856.13)
    
    ts6 = Timestamp()
    

In [7]:
pm = None

class Report(Entity):
    @node(stored=True)
    def valuable(self):
        return None
    
    @node(stored=True)
    def ts1(self):
        return None
    
    @node(stored=True)
    def ts2(self):
        return None
    
    @node
    def data(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        clock = valuable.getObj(_tr.RootClock, 'Main')
    
        def clocks(ts):
            def fn(node):
                obj = node.key[0]
                m = node.key[1].split(':')[-1]
                if isinstance(obj, _tr.Clock) and m == 'cutoffs':
                    return True
            with Context({clock.cutoffs: ts}, 'Clocks'):
                nodes = find(valuable.NPV, fn)
                return dict( [ (node.tweakPoint, node) for node in nodes ] )
    
        allNodes = clocks(ts1)
    
        allNodes.update(clocks(ts2))
        nodes = allNodes.values() 
    
        # IRL, we'd sort these according to some business req...
        nodes = sorted(nodes, key = lambda node: node.key[0].meta.name())
    
        data = []
        curr = [0]
        def add(title, npv):
            pnl = npv - curr[0]
            curr[0] = npv
            data.append( {'Activity': title, 'PnL': pnl } )

        with Context({clock.cutoffs: ts1}, 'Start'):
            curr = [ valuable.NPV() ] # Starting balance
    
        tweaks = {}
        for n in nodes:
            tweaks[n.tweakPoint] = ts1
        with Context(tweaks, name='Start breaks'):
            start = valuable.NPV()
            add('Starting balance breaks', start)

        tsAmend = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
        for n in nodes:
            tweaks[n.tweakPoint] = tsAmend
            name = n.key[0].meta.name()
            with Context(tweaks, name='Amend %s' % name):
                add('prior day amends: %s' % name, valuable.NPV())
        for n in nodes:
            tweaks[n.tweakPoint] = ts2
            name = n.key[0].meta.name()
            with Context(tweaks, name='Activity %s' % name):
                add('activity: %s' % name, valuable.NPV())
    
        with Context({clock.cutoffs: ts2}, name='End'):
            end = valuable.NPV()
            add('Ending balance breaks', end)

        title = 'PnL explain for %s: $%s' % (valuable.meta.name(), end-start)
        return data, title

    def run(self):
        data, title = self.data()
        displayHeader('%s' % title)
        displayListOfDicts(data, names=['Activity', 'PnL'] )
    
r = Report(valuable=pAll, ts1=eod, ts2=ts6)
r.run()

# PnL explain for TopOfTheHouse: $1196.00

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|1200.00
|activity: Portfolio|0.00
|activity: Trading|0.00
|Ending balance breaks|0.00

In [8]:
with db:
    p1.setChildren(p1.children() + [b2])

ts7 = Timestamp()

In [9]:
db3 = db.copy()
p = db3._get(pAll.meta.path())
    
# Note misleading cum times below due to recursion...

with ProfileMonitor(mode='sum'): 
    r = Report(valuable=p, ts1=eod, ts2=ts7)
    r.run()

LogMessage: Oops, book appears multiple times
LogMessage: Oops, book appears multiple times
LogMessage: Oops, book appears multiple times
LogMessage: Oops, book appears multiple times


# PnL explain for TopOfTheHouse: $2216.00

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|1200.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Portfolio:items|33|2,503,195|294|75,854|GetValue
|Portfolio:items|33|2,502,900|5,532|75,845|GetValue/Calc
|Report:data|1|1,481,264|13|1,481,264|GetValue
|Report:data|1|1,481,251|372|1,481,251|GetValue/Calc
|TradingContainer:NPV|11|1,476,015|137|134,183|GetValue
|TradingContainer:NPV|11|1,475,878|1,616|134,170|GetValue/Calc
|Workbook:items|224|1,021,675|1,975|4,561|GetValue
|Workbook:items|220|1,019,700|905,266|4,635|GetValue/Calc
|Root:Clocks|2|505,146|869|252,573|Context
|Portfolio:children|66|265,836|374|4,027|GetValue
|Portfolio:children|33|265,461|134,452|8,044|GetValue/Calc
|Equity:NPV|15|179,113|331|11,940|GetValue
|Equity:NPV|15|178,781|344|11,918|GetValue/Calc
|MarketInterface:spot|15|166,820|125|11,121|GetValue
|MarketInterface:spot|15|166,694|402|11,112|GetValue/Calc
|ExternalRefData:state|15|157,623|126|10,508|GetValue
|ExternalRefData:state|15|157,496|283|10,499|GetValue/Calc
|RefData:state|15|157,212|163|10,480|GetValue
|RefData:state|15|157,049|67,733|10,469|GetValue/Calc
|Clock:cutoffs|544|136,202|1,994|250|GetValue
|Clock:cutoffs|20|134,207|395|6,710|GetValue/Calc
|Root:End|1|125,242|39|125,242|Context
|Root:Start|1|122,199|20|122,199|Context
|Root:Activity Portfolio|1|107,239|36|107,239|Context
|Root:Activity Trading|1|105,363|35|105,363|Context
|Root:Amend MarketData|1|104,871|33|104,871|Context
|Root:Amend Trading|1|104,609|38|104,609|Context
|Root:Start breaks|1|101,909|41|101,909|Context
|Clock:parent|20|101,586|139|5,079|GetValue
|Clock:parent|20|101,446|83,475|5,072|GetValue/Calc
|Root:Amend Portfolio|1|100,919|41|100,919|Context
|Root:Activity MarketData|1|99,709|40|99,709|Context
|TradingBook|22|85,565|85,565|3,889|Db.Get
|PortfolioUpdateEvent:children|33|85,402|496|2,587|GetValue
|RefDataUpdateEvent|9|41,906|41,906|4,656|Db.Get
|Clock|5|21,427|21,427|4,285|Db.Get
|PortfolioUpdateEvent|4|20,926|20,926|5,231|Db.Get
|TradeOpenEvent:ticket|66|19,022|360|288|GetValue
|TradingTicket|4|18,662|18,662|4,665|Db.Get
|TradeOpenEvent|4|17,751|17,751|4,437|Db.Get
|TradingBook:clock|440|12,037|3,868|27|GetValue
|Equity:refdata|15|11,616|133|774|GetValue
|Equity:refdata|15|11,482|435|765|GetValue/Calc
|ClockEvent:parent|8|8,890|67|1,111|GetValue
|MarketInterface:source|15|8,668|116|577|GetValue
|MarketInterface:source|15|8,552|349|570|GetValue/Calc
|ClockEvent|2|8,354|8,354|4,177|Db.Get
|_WorkItemEvent:book2|66|8,288|308|125|GetValue
|TradingBook:clock|220|8,169|3,519|37|GetValue/Calc
|MarketDataSource|2|8,023|8,023|4,011|Db.Get
|_WorkItemEvent:item|33|7,365|216|223|GetValue
|TradingPortfolio|2|7,320|7,320|3,660|Db.Get
|Equity|2|7,148|7,148|3,574|Db.Get
|MarketInterface|2|7,107|7,107|3,553|Db.Get
|Portfolio:clock|66|5,269|583|79|GetValue
|TradeOpenEvent:premium|33|4,886|199|148|GetValue
|ForwardCashflow|1|4,686|4,686|4,686|Db.Get
|Portfolio:clock|33|4,685|481|141|GetValue/Calc
|MarketDataSource:clock|30|4,302|290|143|GetValue
|MarketDataSource:clock|15|4,012|262|267|GetValue/Calc
|MarketInterface|1|3,822|3,822|3,822|Db.Put
|RootClock|1|3,649|3,649|3,649|Db.Get
|Event:amends|142|2,433|2,433|17|GetValue
|Portfolio:books|55|1,751|387|31|GetValue
|Portfolio:books|33|1,363|1,185|41|GetValue/Calc
|Entity:clock|40|534|325|13|GetValue
|_WorkItemEvent:book1|66|323|323|4|GetValue
|TradeOpenEvent:quantity|66|275|275|4|GetValue
|RefDataUpdateEvent:data|58|263|263|4|GetValue
|ForwardCashflow:NPV|11|235|171|21|GetValue
|Entity:clock|20|208|208|10|GetValue/Calc
|MarketInterface:sourceName|15|179|115|11|GetValue
|TradeOpenEvent:action|33|176|176|5|GetValue
|TradeOpenEvent:unitPrice|33|174|174|5|GetValue
|RootClock:cutoffs|52|167|167|3|GetValue
|Equity:assetName|15|117|117|7|GetValue
|MarketInterface:sourceName|15|63|63|4|GetValue/Calc
|ForwardCashflow:NPV|11|63|63|5|GetValue/Calc
|Report:valuable|1|5|5|5|GetValue
|Report:ts2|1|5|5|5|GetValue
|Report:ts1|1|5|5|5|GetValue